In [21]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from os import path
import imblearn
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
import sklearn.preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import Normalizer
import src.lib.utility_classfier as uclf
import src.lib.optimal_threhold_related as thres
import src.lib.fairness_tests as fair

In [22]:
data_path='/Users/lifuchen/Desktop/research/data.csv'
df = pd.read_csv(data_path)

In [23]:
y = df.Class.values
X = df.drop(['GRID','Class','Race_B'], axis=1)
X.shape

(109490, 86)

In [24]:
def save_prediction(classifier, characteristic, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test, X_val_male_scaled, y_val_male, X_test_male_scaled, y_test_male, X_val_female_scaled, y_val_female, X_test_female_scaled, y_test_female):
    method_to_call = getattr(uclf, classifier)
    file_path = '/Users/lifuchen/Desktop/Evaluating-and-Mitigating-Bias-in-ML-Models-for-CVD/Models/'
    filename = str(classifier) + '_' + characteristic[-1] + '_model.sav'
    clf = method_to_call(X_train_scaled, y_train, X_test_scaled, y_test, dump_model=False, file_name = file_path + filename)
    
    y_val_score = clf.predict_proba(X_val_scaled)[:, 1]
    y_test_score = clf.predict_proba(X_test_scaled)[:, 1]
    y_val_score_male = clf.predict_proba(X_val_male_scaled)[:, 1]
    y_test_score_male = clf.predict_proba(X_test_male_scaled)[:, 1]
    y_val_score_female = clf.predict_proba(X_val_female_scaled)[:, 1]
    y_test_score_female = clf.predict_proba(X_test_female_scaled)[:, 1] 
    
    my_dict = dict(val_score = y_val_score, test_score = y_test_score, val_1_score = y_val_score_male, test_1_score = y_test_score_male, val_2_score = y_val_score_female, test_2_score = y_test_score_female)
    overall_prediction = pd.DataFrame.from_dict(my_dict, orient='index')
    overall_prediction = overall_prediction.transpose()

    result_path='/Users/lifuchen/Desktop/research/predictions/'
    filename = str(classifier) + str(characteristic) + "prediction.csv"
    overall_prediction.to_csv(path.join(result_path, filename), index=False)

In [25]:
def get_result (classifier, characteristic, records, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test, X_val_white_scaled, y_val_white, X_test_white_scaled, y_test_white, X_val_black_scaled, y_val_black, X_test_black_scaled, y_test_black):
    result_path='/Users/lifuchen/Desktop/research/predictions/'
    filename = str(classifier) + characteristic + "prediction.csv"
    prediction = pd.read_csv(path.join(result_path, filename))
    
    y_val_score = prediction['val_score'][prediction['val_score'].notna()]
    y_test_score = prediction['test_score'][prediction['test_score'].notna()]
    
    y_val_score_white = prediction['val_1_score'][prediction['val_1_score'].notna()]
    y_test_score_white = prediction['test_1_score'][prediction['test_1_score'].notna()]
    
    y_val_score_black = prediction['val_2_score'][prediction['val_2_score'].notna()]
    y_test_score_black = prediction['test_2_score'][prediction['test_2_score'].notna()]
    
    threshold, ba_val, ba_test = balance_accuracy (y_val, y_val_score,y_test, y_test_score)
    auroc = roc_auc_score(y_test, y_test_score)
    precision, recall, tpr, tnr, pd_overall = thres.calculate_precision_metrics(y_test, y_test_score,threshold)
    
    threshold_white, ba_val_white, ba_test_white = balance_accuracy (y_val_white, y_val_score_white,y_test_white, y_test_score_white)
    precision_white, recall_white, tpr_white, tnr_white, pd_white = thres.calculate_precision_metrics(y_test_white, y_test_score_white,threshold_white)
    
    threshold_black, ba_val_black, ba_test_black = balance_accuracy (y_val_black, y_val_score_black, y_test_black, y_test_score_black)
    precision_black, recall_black, tpr_black, tnr_black, pd_black = thres.calculate_precision_metrics(y_test_black, y_test_score_black,threshold_black)

    eod = fair.get_EOD(y_test_white, y_test_score_white,threshold_white, y_test_black, y_test_score_black, threshold_black)
    sp = fair.get_SP(y_test_white, y_test_score_white,threshold_white, y_test_black, y_test_score_black, threshold_black)

    records.append({
        'auroc': auroc,
        'overall threshold': threshold,
        'white threshold': threshold_white,
        'black threshold': threshold_black,
        'overall ba validation': ba_val,
        'overall ba test': ba_test,
        'white ba validation': ba_val_white,
        'white ba test': ba_test_white,
        'black ba validation': ba_val_black,
        'black ba test': ba_test_black,
        'overall precision':precision,
        'overall recall':recall,
        'overall tpr':tpr,
        'overall tnr':tnr,
        'overall pd':pd_overall,
        'white precision':precision_white,
        'white recall':recall_white,
        'white tpr':tpr_white,
        'white tnr':tnr_white,
        'white pd':pd_white,
        'black precision':precision_black,
        'black recall':recall_black,
        'black tpr':tpr_black,
        'black tnr':tnr_black,
        'black pd':pd_black,
        'eod': eod,
        'di': sp,
        })

In [26]:
def balance_accuracy (y_val, y_val_score,y_test, y_test_score):
    
    threshold, _ = thres.get_optimal_threshold_Jvalue (y_val, y_val_score)
    print ("Optimal threshold by J value is ",threshold)

    ba_val = thres.calculate_balanced_accuracy(y_val, y_val_score, threshold)
    print ("Balanced accuracy score of val is ", ba_val)

    ba_test = thres.calculate_balanced_accuracy(y_test, y_test_score, threshold)
    print ("Balanced accuracy score of test is ",ba_test)

    return threshold, ba_val, ba_test

In [27]:
def fairness_metrics (X, y, attribute, random_state):
    X_train, y_train, X_val, y_val, X_test, y_test, X_val_white, X_val_black, y_val_white, y_val_black, X_test_white, X_test_black, y_test_white, y_test_black \
        = fair.split_by_trait_balance_proportion_no_protected_trait(X, y, attribute, random_state)
    
    print("X train", X_train.shape[0])
    print("Y train", y_train.shape[0])
    print(X_val.shape[0], X_val_white.shape[0], X_val_black.shape[0])
    print(y_val.shape[0], y_val_white.shape[0], y_val_black.shape[0])
    print(X_test.shape[0], X_test_white.shape[0], X_test_black.shape[0])
    print(y_test.shape[0], y_test_white.shape[0], y_test_black.shape[0])

    max_abs_scaler = preprocessing.MaxAbsScaler()
    X_train_scaled = max_abs_scaler.fit_transform(X_train)
    X_test_scaled = max_abs_scaler.transform(X_test)
    X_test_white_scaled = max_abs_scaler.transform(X_test_white)
    X_test_black_scaled = max_abs_scaler.transform(X_test_black)
    X_val_scaled = max_abs_scaler.transform(X_val)
    X_val_white_scaled = max_abs_scaler.transform(X_val_white)
    X_val_black_scaled = max_abs_scaler.transform(X_val_black)

    characteristic = attribute + "resample-by-proportion" + str(random_state)
    #save_prediction ("logic_regression", characteristic, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test, X_val_white_scaled, y_val_white, X_test_white_scaled, y_test_white, X_val_black_scaled, y_val_black, X_test_black_scaled, y_test_black)
    #save_prediction ("random_forest", characteristic, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test, X_val_white_scaled, y_val_white, X_test_white_scaled, y_test_white, X_val_black_scaled, y_val_black, X_test_black_scaled, y_test_black)
    #save_prediction ("decision_tree", characteristic, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test, X_val_white_scaled, y_val_white, X_test_white_scaled, y_test_white, X_val_black_scaled, y_val_black, X_test_black_scaled, y_test_black)
    #save_prediction ("gradiant_boosting", characteristic, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test, X_val_white_scaled, y_val_white, X_test_white_scaled, y_test_white, X_val_black_scaled, y_val_black, X_test_black_scaled, y_test_black)

    get_result ("logic_regression", characteristic, records_lr, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test, X_val_white_scaled, y_val_white, X_test_white_scaled, y_test_white, X_val_black_scaled, y_val_black, X_test_black_scaled, y_test_black)
    get_result ("random_forest", characteristic, records_rf, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test, X_val_white_scaled, y_val_white, X_test_white_scaled, y_test_white, X_val_black_scaled, y_val_black, X_test_black_scaled, y_test_black)
    #get_result ("decision_tree", characteristic, records_dt, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test, X_val_white_scaled, y_val_white, X_test_white_scaled, y_test_white, X_val_black_scaled, y_val_black, X_test_black_scaled, y_test_black)
    get_result ("gradiant_boosting", characteristic, records_gbt, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test, X_val_white_scaled, y_val_white, X_test_white_scaled, y_test_white, X_val_black_scaled, y_val_black, X_test_black_scaled, y_test_black)

In [28]:
records_lr = []
records_rf = []
#records_dt = []
records_gbt = []
for random_state in range(10):
    fairness_metrics (X, y, "Race_W", random_state)

result_lr = pd.DataFrame(records_lr)
result_rf = pd.DataFrame(records_rf)
#result_dt = pd.DataFrame(records_dt)
result_gbt = pd.DataFrame(records_gbt)

/Users/lifuchen/Desktop/Evaluating-and-Mitigating-Bias-in-ML-Models-for-CVD/src/lib/fairness_tests.py:323: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train ['Class'] = y_train


(9055, 87)
(56639, 87)
0.09307098020280058 0.0988475865280149
0.0987445678416224
(65741, 86)
X train 65741
Y train 65741
21898 18899 2999
21898 18899 2999
21898 18968 2930
21898 18968 2930
threshold:0.0, J-value:0.0
threshold:0.1, J-value:0.41400000000000003
threshold:0.2, J-value:0.26799999999999996
threshold:0.30000000000000004, J-value:0.152
threshold:0.4, J-value:0.07100000000000001
threshold:0.5, J-value:0.029
threshold:0.6000000000000001, J-value:0.011
threshold:0.7000000000000001, J-value:0.001
threshold:0.8, J-value:0.0
threshold:0.9, J-value:0.0
Optimal threshold by J value is  0.1
Balanced accuracy score of val is  0.7067594715416345
Balanced accuracy score of test is  0.6986706845420846
threshold:0.0, J-value:0.0
threshold:0.1, J-value:0.40700000000000003
threshold:0.2, J-value:0.25899999999999995
threshold:0.30000000000000004, J-value:0.15
threshold:0.4, J-value:0.067
threshold:0.5, J-value:0.028
threshold:0.6000000000000001, J-value:0.009999999999999998
threshold:0.7000000

/Users/lifuchen/Desktop/Evaluating-and-Mitigating-Bias-in-ML-Models-for-CVD/src/lib/fairness_tests.py:323: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train ['Class'] = y_train


(8896, 87)
(56798, 87)
0.09772951628825272 0.10052315442743655
0.10044422507403751
(65716, 86)
X train 65716
Y train 65716
21898 18825 3073
21898 18825 3073
21898 18883 3015
21898 18883 3015
threshold:0.0, J-value:0.0
threshold:0.1, J-value:0.41000000000000003
threshold:0.2, J-value:0.27299999999999996
threshold:0.30000000000000004, J-value:0.154
threshold:0.4, J-value:0.07100000000000001
threshold:0.5, J-value:0.033
threshold:0.6000000000000001, J-value:0.013000000000000001
threshold:0.7000000000000001, J-value:0.004
threshold:0.8, J-value:0.001
threshold:0.9, J-value:0.0
Optimal threshold by J value is  0.1
Balanced accuracy score of val is  0.7047020691181275
Balanced accuracy score of test is  0.6975687301786364
threshold:0.0, J-value:0.0
threshold:0.1, J-value:0.40700000000000003
threshold:0.2, J-value:0.27799999999999997
threshold:0.30000000000000004, J-value:0.153
threshold:0.4, J-value:0.066
threshold:0.5, J-value:0.029
threshold:0.6000000000000001, J-value:0.012
threshold:0.70

/Users/lifuchen/Desktop/Evaluating-and-Mitigating-Bias-in-ML-Models-for-CVD/src/lib/fairness_tests.py:323: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train ['Class'] = y_train


...

(8973, 87)
(56721, 87)
0.09320175438596491 0.0980099887723102
0.097953216374269
(65733, 86)
X train 65733
Y train 65733
21898 18920 2978
21898 18920 2978
21898 18865 3033
21898 18865 3033
threshold:0.0, J-value:0.0
threshold:0.1, J-value:0.41200000000000003
threshold:0.2, J-value:0.26899999999999996
threshold:0.30000000000000004, J-value:0.145
threshold:0.4, J-value:0.068
threshold:0.5, J-value:0.027
threshold:0.6000000000000001, J-value:0.006999999999999999
threshold:0.7000000000000001, J-value:0.004
threshold:0.8, J-value:0.0
threshold:0.9, J-value:0.0
Optimal threshold by J value is  0.1
Balanced accuracy score of val is  0.7059787945307896
Balanced accuracy score of test is  0.6992024718052959
threshold:0.0, J-value:0.0
threshold:0.1, J-value:0.41500000000000004
threshold:0.2, J-value:0.27299999999999996
threshold:0.30000000000000004, J-value:0.146
threshold:0.4, J-value:0.068
threshold:0.5, J-value:0.027
threshold:0.6000000000000001, J-value:0.008
threshold:0.7000000000000001, J-v

/Users/lifuchen/Desktop/Evaluating-and-Mitigating-Bias-in-ML-Models-for-CVD/src/lib/fairness_tests.py:323: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train ['Class'] = y_train


(9011, 87)
(56683, 87)
0.09316996239233288 0.09987193418193108
0.09984229042824215
(65749, 86)
X train 65749
Y train 65749
21898 18905 2993
21898 18905 2993
21898 18918 2980
21898 18918 2980
threshold:0.0, J-value:0.0
threshold:0.1, J-value:0.393
threshold:0.2, J-value:0.256
threshold:0.30000000000000004, J-value:0.149
threshold:0.4, J-value:0.069
threshold:0.5, J-value:0.036000000000000004
threshold:0.6000000000000001, J-value:0.008
threshold:0.7000000000000001, J-value:0.003
threshold:0.8, J-value:0.001
threshold:0.9, J-value:0.0
Optimal threshold by J value is  0.1
Balanced accuracy score of val is  0.6965624694468922
Balanced accuracy score of test is  0.6999288754518437
threshold:0.0, J-value:0.0
threshold:0.1, J-value:0.391
threshold:0.2, J-value:0.25
threshold:0.30000000000000004, J-value:0.15
threshold:0.4, J-value:0.069
threshold:0.5, J-value:0.034999999999999996
threshold:0.6000000000000001, J-value:0.007
threshold:0.7000000000000001, J-value:0.002
threshold:0.8, J-value:0.00

/Users/lifuchen/Desktop/Evaluating-and-Mitigating-Bias-in-ML-Models-for-CVD/src/lib/fairness_tests.py:323: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train ['Class'] = y_train


(8852, 87)
(56842, 87)
0.09297444128904803 0.09886328487472935
0.09877762686751451
(65741, 86)
X train 65741
Y train 65741
21898 18847 3051
21898 18847 3051
21898 18817 3081
21898 18817 3081
threshold:0.0, J-value:0.0
threshold:0.1, J-value:0.40800000000000003
threshold:0.2, J-value:0.263
threshold:0.30000000000000004, J-value:0.154
threshold:0.4, J-value:0.07500000000000001
threshold:0.5, J-value:0.032
threshold:0.6000000000000001, J-value:0.011
threshold:0.7000000000000001, J-value:0.004
threshold:0.8, J-value:0.001
threshold:0.9, J-value:0.0
Optimal threshold by J value is  0.1
Balanced accuracy score of val is  0.7039909202098158
Balanced accuracy score of test is  0.708147863322053
threshold:0.0, J-value:0.0
threshold:0.1, J-value:0.40800000000000003
threshold:0.2, J-value:0.261
threshold:0.30000000000000004, J-value:0.151
threshold:0.4, J-value:0.07
threshold:0.5, J-value:0.030000000000000002
threshold:0.6000000000000001, J-value:0.012
threshold:0.7000000000000001, J-value:0.004


In [29]:
def add_mean_sd(records, result_table, overall_records, type):
    records.append({
        'auroc': result_table["auroc"].mean(),
        'overall threshold': result_table["overall threshold"].mean(),
        'white threshold': result_table["white threshold"].mean(),
        'black threshold': result_table["black threshold"].mean(),
        'overall ba validation': result_table["overall ba validation"].mean(),
        'overall ba test': result_table["overall ba test"].mean(),
        'white ba validation': result_table["white ba validation"].mean(),
        'white ba test': result_table["white ba test"].mean(),
        'black ba validation': result_table["black ba validation"].mean(),
        'black ba test': result_table["black ba test"].mean(),
        'overall precision':result_table["overall precision"].mean(),
        'overall recall':result_table["overall recall"].mean(),
        'overall tpr':result_table["overall tpr"].mean(),
        'overall tnr':result_table["overall tnr"].mean(),
        'overall pd':result_table["overall pd"].mean(),
        'white precision':result_table["white precision"].mean(),
        'white recall':result_table["white recall"].mean(),
        'white tpr':result_table["white tpr"].mean(),
        'white tnr':result_table["white tnr"].mean(),
        'white pd':result_table["white pd"].mean(),
        'black precision':result_table["black precision"].mean(),
        'black recall':result_table["black recall"].mean(),
        'black tpr':result_table["black tpr"].mean(),
        'black tnr':result_table["black tnr"].mean(),
        'black pd':result_table["black pd"].mean(),
        'eod': result_table["eod"].mean(),
        'di': result_table["di"].mean(),
        })
    records.append({
        'auroc': result_table["auroc"].std(),
        'overall threshold': result_table["overall threshold"].std(),
        'white threshold': result_table["white threshold"].std(),
        'black threshold': result_table["black threshold"].std(),
        'overall ba validation': result_table["overall ba validation"].std(),
        'overall ba test': result_table["overall ba test"].std(),
        'white ba validation': result_table["white ba validation"].std(),
        'white ba test': result_table["white ba test"].std(),
        'black ba validation': result_table["black ba validation"].std(),
        'black ba test': result_table["black ba test"].std(),
        'overall precision':result_table["overall precision"].std(),
        'overall recall':result_table["overall recall"].std(),
        'overall tpr':result_table["overall tpr"].std(),
        'overall tnr':result_table["overall tnr"].std(),
        'overall pd':result_table["overall pd"].std(),
        'white precision':result_table["white precision"].std(),
        'white recall':result_table["white recall"].std(),
        'white tpr':result_table["white tpr"].std(),
        'white tnr':result_table["white tnr"].std(),
        'white pd':result_table["white pd"].std(),
        'black precision':result_table["black precision"].std(),
        'black recall':result_table["black recall"].std(),
        'black tpr':result_table["black tpr"].std(),
        'black tnr':result_table["black tnr"].std(),
        'black pd':result_table["black pd"].std(),
        'eod': result_table["eod"].std(),
        'di': result_table["di"].std(),
        })
    overall_records.append({
        'type': type,
        'auroc': result_table["auroc"].mean(),
        'auroc_std': result_table["auroc"].std(),
        'overall threshold': result_table["overall threshold"].mean(),
        'white threshold': result_table["white threshold"].mean(),
        'black threshold': result_table["black threshold"].mean(),
        'overall ba test': result_table["overall ba test"].mean(),
        'ba_std': result_table["overall ba test"].std(),
        'white ba test': result_table["white ba test"].mean(),
        'black ba test': result_table["black ba test"].mean(),
        'overall tpr':result_table["overall tpr"].mean(),
        'overall pd':result_table["overall pd"].mean(),
        'white tpr':result_table["white tpr"].mean(),
        'white pd':result_table["white pd"].mean(),
        'black tpr':result_table["black tpr"].mean(),
        'black pd':result_table["black pd"].mean(),
        'eod': result_table["eod"].mean(),
        'eod_std': result_table["eod"].std(),
        'di': result_table["di"].mean(),
        'di_std': result_table["di"].std()
        })
    pd_result = pd.DataFrame(records)
    return pd_result, overall_records

In [30]:
overall_table = []
result_lr, overall_records = add_mean_sd (records_lr, result_lr, overall_table, 'lr')
result_rf, overall_records = add_mean_sd (records_rf, result_rf, overall_records, 'rf')
#result_dt, overall_records = add_mean_sd (records_dt, result_dt, overall_records, 'dt')
result_gbt, overall_records = add_mean_sd (records_gbt, result_gbt, overall_records, 'gbt')

result_path='/Users/lifuchen/Desktop/research/resample_data/'
result_lr.to_csv(path.join(result_path,'race-lr-resample-proportion-result.csv'), index=False)
result_rf.to_csv(path.join(result_path,'race-rf-resample-proportion-result.csv'), index=False)
#result_dt.to_csv(path.join(result_path,'race-dt-resample-proportion-result.csv'), index=False)
result_gbt.to_csv(path.join(result_path,'race-gbt-resample-proportion-result.csv'), index=False)

overall_result = pd.DataFrame(overall_table)
result_path='/Users/lifuchen/Desktop/research/resample_result/'
overall_result.to_csv(path.join(result_path,'race-resample-proportion.csv'), index=False)